### Tên : Huỳnh Ngọc Công Danh
### MSSV : 19521322

In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import StandardScaler,OneHotEncoder
df = pd.read_csv("/content/Churn_Modelling.csv")

df.dropna()
df.head(1)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,101348.88,1


In [2]:
dr = ['RowNumber','CustomerId','Surname']
df = df.drop(dr,axis=1)
df.head(1)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.0,1,1,1,101348.88,1


In [3]:
# Sử dụng onehot
X = df.iloc[:,:-1]
y = df.iloc[:, -1]
columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), ['Gender','Geography'])],remainder='passthrough') 
X_one_hot = pd.DataFrame(columnTransformer.fit_transform(X))
X_one_hot.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.0,0.0,1.0,0.0,0.0,619.0,42.0,2.0,0.0,1.0,1.0,1.0,101348.88


In [4]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test = train_test_split(X_one_hot,y,test_size=0.2,random_state=42)
scale = StandardScaler()
x_scale = scale.fit_transform(x_train)
x_test = scale.fit_transform(x_test)

In [5]:
### Xây dựng mạng neural
from keras.models import Sequential,Model
from keras.layers import Activation, Dense
def train_model():
  model = Sequential()
  model.add(Dense(6, activation='relu'))
  model.add(Dense(6,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',
              optimizer='adam',metrics=['accuracy'])
  return model
# model.layers

In [6]:
# model.compile(loss='binary_crossentropy',
#               optimizer='adam',metrics=['accuracy'])
model_trained = train_model()
model_trained.fit(x_scale, y_train,
          batch_size=32,
          epochs=100
          )

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.6277 - accuracy: 0.6775
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4926 - accuracy: 0.7947
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4574 - accuracy: 0.7983
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4426 - accuracy: 0.8054
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4329 - accuracy: 0.8066
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4241 - accuracy: 0.8146
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4166 - accuracy: 0.8202
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4101 - accuracy: 0.8217
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4038 - accuracy: 0.8249
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3959 - accura

In [7]:
print("Evaluate on test data")
_, acc = model_trained.evaluate(x_test, y_test, batch_size=32)
print("accurac_score :", acc)

Evaluate on test data
63/63 [==============================] - 0s 1ms/step - loss: 0.3349 - accuracy: 0.8635
accurac_score : 0.8634999990463257


In [61]:
### sử dụng kflod 
test_data = pd.read_csv("test.csv")
X_add = X.append(test_data)
X_add_one = columnTransformer.fit_transform(X_add)

X_test = scale.fit_transform(X_add_one[10000:])
predictions = model_trained.predict(X_test)
print(" Ke qua du doan ra:", round(predictions[0][0]))

 Ke qua du doan ra: 0


In [17]:
# Sử dụng kfold
from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score
def Score(model,X,y):
  acc_score = [] 
  k = 5
  kf = KFold(n_splits=k,shuffle=True)
  for train_index , test_index in kf.split(X):
    X_train , X_test = X[train_index],X[test_index]
    y_train , y_test =y[train_index] , y[test_index]
    X_scaled_train = scale.fit_transform(X_train)
    X_scaled_test = scale.fit_transform(X_test)
    model.fit(x_scale, y_train,
          batch_size=32,
          epochs=100
          )
    
    _, acc = model.evaluate(x_test, y_test, batch_size=32)
    acc_score.append(acc)
  return sum(acc_score)/len(acc_score)


In [21]:
model_trained_2 = train_model()
print(model_trained_2)
score = Score(model_trained_2,np.array(X_one_hot),np.array(y))


Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5401 - accuracy: 0.7941
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5173 - accuracy: 0.7961
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5118 - accuracy: 0.7961
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5093 - accuracy: 0.7961
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5079 - accuracy: 0.7961
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5071 - accuracy: 0.7961
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5066 - accuracy: 0.7961
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5061 - accuracy: 0.7961
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5056 - accuracy: 0.7961
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5056 - accura

In [22]:
print("Kfold {}".format(score))

Kfold 0.7958000063896179
